In [3]:
%matplotlib inline

import numpy as np
import pandas as pd
from collections import OrderedDict
import matplotlib.pyplot as plt
import math

In [10]:
class FCT:

    def __init__(self, folder, toponame, flownum, loads, algs):
        self.flownum = flownum
        self.loads = loads
        self.algs = algs
        self.raw_data = [[0 for i in range(len(loads))] for i in range(len(algs))]
        for alg_i in range(len(algs)):
            for load_i in range(len(loads)):
                alg = algs[alg_i]
                load = loads[load_i]
                self.raw_data[alg_i][load_i] = \
                    self.parse_trace(folder, filename="tcp_"+alg+"_flow_"+str(flownum)+"_"+str(load)+"_"+toponame+".tr")
                    
    # At least two
    def plotting_mean_FCT(self):
        plt.figure(figsize=(8,5))
        plt.xlabel("Link utilization")
        plt.ylabel("Normalized FCT")
        plt.title("Normalized FCT")
        for alg_i in range(len(self.algs)):
            alg = self.algs[alg_i]
            load_means = [0 for i in range(len(self.loads))]
            for load_i in range(len(self.loads)):
                load = self.loads[load_i]
                load_means[load_i] = self.raw_data[alg_i][load_i]["Normalized FCT"].mean()
            plt.plot(self.loads, load_means, label=alg)
        plt.legend(loc='best')
        plt.show()
    
    # Normalized FCT = Actual FCT / min FCT
    # min FCT = flow_size / max bandwidth
    def parse_trace(self, dir, filename):
        col = ["tmp_pkts", "flow duration", "fin_fid", "rtt times", \
                   "group_id", "Tw_", "Tp_", "Np_", "?1", "?2", "?3", "?4", "?5", "Average rate", "start time"]
        df = pd.read_csv(dir+filename, sep = ' ', header = None)
        df.columns = col
        df["Normalized FCT"] = 40 * 1e9 / df["Average rate"]
        return df
    
    def get_load_id(self, load):
        return self.loads.index(load)
    
    def get_alg_id(self, alg):
        return self.algs.index(alg)
    
    def humanbytes(self, B):
        B = float(B)
        KB = float(1024)
        MB = float(KB ** 2) # 1,048,576
        GB = float(KB ** 3) # 1,073,741,824
        TB = float(KB ** 4) # 1,099,511,627,776

        if B < KB:
            return '{0} {1}'.format(B,'Bytes' if 0 == B > 1 else 'Byte')
        elif KB <= B < MB:
            return '{0:.2f} KB'.format(B/KB)
        elif MB <= B < GB:
            return '{0:.2f} MB'.format(B/MB)
        elif GB <= B < TB:
            return '{0:.2f} GB'.format(B/GB)
        elif TB <= B:
            return '{0:.2f} TB'.format(B/TB)
        
    def plot_FCT_flowsize(self, alg, load, color=None, scale=[0,0], label=None):
        alg_id=self.get_alg_id(alg)
        load_id=self.get_load_id(load)
        df=self.raw_data[alg_id][load_id]
        y = df['Normalized FCT'].apply(self.scaling, args=(scale[1],))
        x = df['tmp_pkts'].apply(self.scaling, args=(scale[0],))
        plt.scatter(x, y, color=color, label=label)
        plt.legend()
        plt.show()
    def scaling(self, x, base):
        return x if (base == 0 or x<base) else base + x / base
    
    def plot_one_FCT_percentile(self, alg, load, percentile, flow_size, scale=[0,0], isLast=False):
        alg_id=self.get_alg_id(alg)
        load_id=self.get_load_id(load)
        df = self.raw_data[alg_id][load_id]
        # cut -> (,]
        cate=pd.cut(df["tmp_pkts"], flow_size, labels=flow_size[:-1])
        selected_flow=[]
        corresponding_flow_size=[]
        for label in flow_size[:-1]:
            seg_data=df[cate == label]
            res=seg_data.sort_values(by=['Normalized FCT'])
            res.reset_index(inplace=True)
            # 16 indicate col 'Normalized FCT' 
            print(res)
            #print(res.iloc[[math.ceil(res.shape[0] * 0.95)]])
            selected_flow.append(res.iloc[math.ceil(res.shape[0] * 0.95), 16])
        scaled_selected_flow = [self.scaling(x, scale[0]) for x in selected_flow]
        scaled_flow_size = [self.scaling(x, scale[1]) for x in flow_size[:-1]]
        plt.plot(scaled_flow_size, scaled_selected_flow, label=alg)
        if (isLast):
            plt.xlabel('flow size')
            plt.ylabel('Normalized FCT')
            plt.title("load:"+str(load)+" percentile:"+str(percentile))
            plt.grid()
            plt.legend()
            #plt.yticks([0,50,100,150,200])
            #plt.xticks(scaled_flow_size, [self.humanbytes(i*1500) for i in flow_size[:-1]])
            plt.show()
            
    def plot_FCT_percentile_fix_load(self, algs, load, percentile, flow_size, scale):
        for alg in algs:
            if alg == algs[-1]:
                FCT_analysis.plot_one_FCT_percentile(alg , load, percentile, flow_size, scale, True)
            else:
                FCT_analysis.plot_one_FCT_percentile(alg , load, percentile, flow_size, scale, False)

In [11]:
FCT_analysis = FCT("/home/ovs2/lining/Gearbox/automatic/", "Topology-incast-singlenode.tcl", 500, \
                  [.1, .2, .3, .4, .5, .6, .7, .8, .9, .95], ["AFQ10", "PQ", "HRCC", "DropTail", "AFQ100", "AFQ1000"])

In [12]:
# Byte
flow_size=[0, 6, 13, 19, 33, 53, 133, 1333, 3333, 6667, float('inf')]
algs=["AFQ10", "PQ", "HRCC", "DropTail", "AFQ100", "AFQ1000"]
FCT_analysis.plot_FCT_percentile_fix_load(algs, .3, .95, flow_size, [200, 50])
FCT_analysis.plot_FCT_percentile_fix_load(algs, .3, .5, flow_size, [200, 50])
FCT_analysis.plot_FCT_percentile_fix_load(algs, .5, .95, flow_size, [200, 50])
FCT_analysis.plot_FCT_percentile_fix_load(algs, .5, .95, flow_size, [200, 50])

    index  tmp_pkts  flow duration  fin_fid  rtt times  group_id  Tw_  Tp_  \
0     281       6.0       0.000051      566          0         0    0    0   
1     239       6.0       0.000051      478          0         0    0    0   
2     245       6.0       0.000051      458          0         0    1    1   
3     276       6.0       0.000051      560          0         0    0    0   
4     496       6.0       0.000051      988          0         0    1    0   
5     286       6.0       0.000051      578          0         0    1    0   
6     304       6.0       0.000051      608          0         0    0    1   
7     323       6.0       0.000051      656          0         0    0    0   
8     325       6.0       0.000051      612          0         0    1    1   
9     327       6.0       0.000051      658          0         0    0    0   
10    339       6.0       0.000051      670          0         0    1    0   
11    341       6.0       0.000051      694          0         0

IndexError: single positional indexer is out-of-bounds

In [ ]:
FCT_analysis.plotting_mean_FCT()

In [ ]:
for alg in algs:
    FCT_analysis.plot_FCT_flowsize(alg, 0.5, label=alg, scale=[0, 50])